# Credit Risk — PD Modeling (J.P. Morgan prototype)
- Notebook author: Jaime (Data Scientist)
- Dataset: Task 3 and 4_Loan_Data.csv
- Reference: CREDIT RISK ANALYSIS TASK.docx.

In [1]:
# ## 0. Setup and imports
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report, precision_recall_curve)
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# xgboost
from xgboost import XGBClassifier



## 1. Load data and quick inspection


In [3]:
# Adjust path if needed
DATA_PATH = 'Task 3 and 4_Loan_Data.csv'
df = pd.read_csv(DATA_PATH)

print("Rows, columns:", df.shape)
df.head()

Rows, columns: (10000, 8)


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [4]:
# Quick datatype/missingness summary
display(df.info())
display(df.describe().T)
display(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               10000 non-null  int64  
 1   credit_lines_outstanding  10000 non-null  int64  
 2   loan_amt_outstanding      10000 non-null  float64
 3   total_debt_outstanding    10000 non-null  float64
 4   income                    10000 non-null  float64
 5   years_employed            10000 non-null  int64  
 6   fico_score                10000 non-null  int64  
 7   default                   10000 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 625.1 KB


None

,count,mean,std,min,25%,50%,75%,max
customer_id,10000.0,4.974577e+06,2.293890e+06,1.000324e+06,2.977661e+06,4.989502e+06,6.967210e+06,8.999789e+06
credit_lines_outstanding,10000.0,1.461200e+00,1.743846e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,5.000000e+00
loan_amt_outstanding,10000.0,4.159677e+03,1.421399e+03,4.678397e+01,3.154235e+03,4.052377e+03,5.052898e+03,1.075068e+04
total_debt_outstanding,10000.0,8.718917e+03,6.627165e+03,3.165273e+01,4.199836e+03,6.732407e+03,1.127226e+04,4.368878e+04
income,10000.0,7.003990e+04,2.007221e+04,1.000000e+03,5.653987e+04,7.008583e+04,8.342917e+04,1.484122e+05
years_employed,10000.0,4.552800e+00,1.566862e+00,0.000000e+00,3.000000e+00,5.000000e+00,6.000000e+00,1.000000e+01
fico_score,10000.0,6.375577e+02,6.065791e+01,4.080000e+02,5.970000e+02,6.380000e+02,6.790000e+02,8.500000e+02
default,10000.0,1.851000e-01,3.883981e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


customer_id                 0
credit_lines_outstanding    0
loan_amt_outstanding        0
total_debt_outstanding      0
income                      0
years_employed              0
fico_score                  0
default                     0
dtype: int64

## 2. ETL / Data cleaning
- Drop `customer_id`.
- Impute numeric missing values with median (if any).
- Basic sanity checks: non-negative income, reasonable FICO (300-850).


In [5]:
# Drop customer id if present
if 'customer_id' in df.columns:
    df = df.drop(columns=['customer_id'])

# Sanity checks
print("FICO min/max:", df['fico_score'].min(), df['fico_score'].max())
print("Income min:", df['income'].min())

# If any negative or zero incomes, flag or impute
df = df[df['income'] > 0].copy()  # keep rows with positive income


FICO min/max: 408 850
Income min: 1000.0


## 3. Feature engineering
Create:
- debt_to_income = total_debt_outstanding / income
- loan_to_income = loan_amt_outstanding / income
- fico_bucket (categorical)


In [ ]:
df['debt_to_income'] = df['total_debt_outstanding'] / df['income']
df['loan_to_income'] = df['loan_amt_outstanding'] / df['income']

# FICO bucket
def fico_bucket(f):
    if f < 580:
        return 'deep_subprime'
    elif f < 640:
        return 'subprime'
    elif f < 700:
        return 'near_prime'
    else:
        return 'prime'
df['fico_bucket'] = df['fico_score'].apply(fico_bucket)

# Quick check
df[['fico_score','fico_bucket','debt_to_income','loan_to_income']].head()


,fico_score,fico_bucket,debt_to_income,loan_to_income
0,605,subprime,0.050173,0.066909
1,572,deep_subprime,0.308789,0.073510
2,602,subprime,0.030787,0.051058
3,612,subprime,0.033645,0.064105
4,631,subprime,0.075435,0.057395


## 4. Prepare features and split data
- Use numeric features: credit_lines_outstanding, loan_amt_outstanding, total_debt_outstanding,
  income, years_employed, fico_score, debt_to_income, loan_to_income
- Encode fico_bucket if needed (one-hot) — I'll use numeric features only for tree models (they don't need scaling).
- Stratified train/test split on `default`.